# MVP - JSON in sqlite

Instead of shoehorning JSONs into a traditional relational DB, store as-is, and leverage the JSON features to get our delay data

1. call aviationstack API
2. save json response to sqlite3 db
3. extract data from response
4. tweet data

In [1]:
import os
import requests
from urllib3.util import Retry
from requests import Session, HTTPError
from requests.adapters import HTTPAdapter
from requests.exceptions import ReadTimeout
import sqlite3
import json
from pathlib import Path
from dotenv import load_dotenv
from datetime import datetime, timezone, timedelta
from time import sleep
import tweepy
import logging
from sys import stdout
import tomllib
import jinja2

In [7]:
logging.basicConfig(
    format="%(asctime)s [%(levelname)s] %(funcName)s: %(message)s",
    datefmt="%Y/%m/%d %H:%M:%S",
    handlers=[logging.StreamHandler(stdout)],
)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

In [2]:
secrets = Path("../secret")
avi_env = secrets / "av1_api"
twitter_env = secrets / "twitter_api"
load_dotenv(avi_env)
load_dotenv(twitter_env)
AV_API_KEY = os.getenv("AVIATION_API_KEY", "")
AV_API_URL = "http://api.aviationstack.com/v1/"
FLIGHT_API_URL = AV_API_URL + "flights"
TWITTER_API_KEY = os.getenv("TWITTER_API_KEY", "")
TWITTER_API_SECRET = os.getenv("TWITTER_API_SECRET", "")
TWITTER_ACCESS_TOKEN = os.getenv("TWITTER_ACCESS_TOKEN", "")
TWITTER_ACCESS_SECRET = os.getenv("TWITTER_ACCESS_SECRET", "")

In [3]:
toml_path = Path("../pyproject.toml")
with open(toml_path, "rb") as f:
    config = tomllib.load(f)

DB_NAME = config["sqlite"]["db_name"]
TBL_NAME = config["sqlite"]["tbl_name"]
JSON_COL = config["sqlite"]["json_col"]

In [4]:
def write_local_json(
    api_response: dict,
    json_dir: Path,
    str_date: str = str(datetime.now(tz=timezone.utc).date() - timedelta(days=1)),
    offset: int = 0,
    limit: int = 100,
):
    """
    Saves the flight api response as json, to be uploaded to a data lake
    json will be named according to the UTC date of when it was retrieved
    """
    if not json_dir.exists():
        json_dir.mkdir(parents=True)
    local_json_path = json_dir / f"flight-{str_date}-{offset}-{offset+limit}.json"
    logger.info(f"saving to {local_json_path}")
    with open(local_json_path, "w") as j:
        json.dump(api_response, j)
        logger.debug(f"saved to {local_json_path}")
    return local_json_path

In [5]:
def get_all_delays(
    str_date: str,
    json_dir: str,
    limit: int = 100,
    airline: str = "Malaysia Airlines",
):
    sesh = Session()
    adapter = HTTPAdapter(
        max_retries=Retry(
            total=3,
            backoff_factor=0.1,
            status_forcelist=[500, 502, 503, 504],
            # allowed_methods={"POST"},
        )
    )
    sesh.mount(AV_API_URL, adapter)
    responses = []
    retrieved = total = 0
    logger.info(f"Retrieving delayed flights for {str_date}")
    while not total or retrieved < total:
        sleep(0.5)
        logger.info(f"retrieving {retrieved}th to {retrieved + limit}th")
        params = {
            "access_key": AV_API_KEY,  # retrieved from .env, global scope
            "offset": retrieved,
            "limit": limit,
            "airline_name": airline,
            # "min_delay_arr": min_delay,
        }
        try:
            response = sesh.get(
                url=FLIGHT_API_URL,
                params=params,
                timeout=30.0,
            )
            response.raise_for_status()
        except HTTPError as exc:
            logger.error(f"HTTP Error: \n{exc}")

        except ReadTimeout as e:
            logger.error(
                f"Timeout retrieving {retrieved}th to {retrieved + limit}th:\n{e}"
            )
        # save response
        response = response.json()
        logger.debug(f"retrieved {retrieved}th to {retrieved + limit}th")
        json_path = write_local_json(
            response, json_dir=json_dir, str_date=str_date, offset=retrieved
        )
        responses.extend(response["data"])
        retrieved += response["pagination"]["count"]
        if not total:
            # First request; get total count
            total = response["pagination"]["total"]
            logger.info(f"Total records count: {total}")
            if total == 0:
                # prevent infinite loop if there are no records retrieved
                logger.error("Zero records retrieved; exiting")
                break
    return responses

## 1. Fetch the responses

In [9]:
str_date = "2023-12-20"
responses = get_all_delays(str_date=str_date, json_dir=Path("../data/mh/responses/"))

2023/12/21 16:49:17 [INFO] get_all_delays: Retrieving delayed flights for 2023-12-20
2023/12/21 16:49:17 [INFO] get_all_delays: retrieving 0th to 100th
2023/12/21 16:49:18 [ERROR] get_all_delays: HTTP Error: 
401 Client Error: Unauthorized for url: http://api.aviationstack.com/v1/flights?access_key=&offset=0&limit=100&airline_name=Malaysia+Airlines
2023/12/21 16:49:18 [DEBUG] get_all_delays: retrieved 0th to 100th
2023/12/21 16:49:18 [INFO] write_local_json: saving to ../data/mh/responses/flight-2023-12-20-0-100.json
2023/12/21 16:49:18 [DEBUG] write_local_json: saved to ../data/mh/responses/flight-2023-12-20-0-100.json


KeyError: 'data'

In [12]:
responses[-1]

{'flight_date': '2023-10-23',
 'flight_status': 'landed',
 'departure': {'airport': 'Dallas/Fort Worth International',
  'timezone': 'America/Chicago',
  'iata': 'DFW',
  'icao': 'KDFW',
  'terminal': 'C',
  'gate': 'C19',
  'delay': 19,
  'scheduled': '2023-10-23T18:39:00+00:00',
  'estimated': '2023-10-23T18:39:00+00:00',
  'actual': '2023-10-23T18:57:00+00:00',
  'estimated_runway': '2023-10-23T18:57:00+00:00',
  'actual_runway': '2023-10-23T18:57:00+00:00'},
 'arrival': {'airport': 'Charlotte Douglas',
  'timezone': 'America/New_York',
  'iata': 'CLT',
  'icao': 'KCLT',
  'terminal': '3',
  'gate': 'D8',
  'baggage': 'E',
  'delay': None,
  'scheduled': '2023-10-23T22:13:00+00:00',
  'estimated': '2023-10-23T22:13:00+00:00',
  'actual': '2023-10-23T21:55:00+00:00',
  'estimated_runway': '2023-10-23T21:55:00+00:00',
  'actual_runway': '2023-10-23T21:55:00+00:00'},
 'airline': {'name': 'Malaysia Airlines', 'iata': 'MH', 'icao': 'MAS'},
 'flight': {'number': '9527',
  'iata': 'MH9527'

In [7]:
# get existing json, for testing
responses = []
str_date = "2023-10-13"
json_dir = Path("../data/responses")
json_paths = json_dir.glob(f"flight-{str_date}-*.json")
for json_file in json_paths:
    logger.debug(f"looking for {json_file}")
    with open(json_file) as j:
        flight_page = json.load(j)
        responses.extend(flight_page["data"])

logger.info(f"{len(responses)} entries on {str_date}")

2023/10/18 10:37:34 [DEBUG] <module>: looking for ../data/responses/flight-2023-10-13-700-800.json
2023/10/18 10:37:35 [DEBUG] <module>: looking for ../data/responses/flight-2023-10-13-800-900.json
2023/10/18 10:37:35 [DEBUG] <module>: looking for ../data/responses/flight-2023-10-13-400-500.json
2023/10/18 10:37:35 [DEBUG] <module>: looking for ../data/responses/flight-2023-10-13-100-200.json
2023/10/18 10:37:35 [DEBUG] <module>: looking for ../data/responses/flight-2023-10-13-300-400.json
2023/10/18 10:37:35 [DEBUG] <module>: looking for ../data/responses/flight-2023-10-13-600-700.json
2023/10/18 10:37:35 [DEBUG] <module>: looking for ../data/responses/flight-2023-10-13-200-300.json
2023/10/18 10:37:35 [DEBUG] <module>: looking for ../data/responses/flight-2023-10-13-0-100.json
2023/10/18 10:37:35 [DEBUG] <module>: looking for ../data/responses/flight-2023-10-13-500-600.json
2023/10/18 10:37:35 [INFO] <module>: 819 entries on 2023-10-13


In [13]:
def execute_template_sql(
    db_conn: sqlite3.Connection,
    env: jinja2.Environment,
    template: str,
    params: dict,
    data: list = None,
):
    """
    Renders the jinja templated sql and executes,
    returning results if any
    """
    sql = env.get_template(template).render(params)
    logger.debug(f"rendered SQL:\n{sql}")
    with db_conn:
        if data:
            db_conn.executemany(sql, data)
            return None
        else:
            # executescript did not return any results
            if "CREATE" in sql or "INSERT INTO" in sql:
                db_conn.executescript(sql)
            else:
                return db_conn.execute(sql)

## 2. Insert JSON to sqlite

In [14]:
# app args
data_dir = Path("../data")
template_dir = Path("../templates")
params = dict(
    tbl_name=TBL_NAME,
    json_col=JSON_COL,
)
# instantiate db conn and jinja env
env = jinja2.Environment(loader=jinja2.FileSystemLoader(template_dir))
# search for existing db
# db_path = data_dir / f"{DB_NAME}.db"
db_path = data_dir / "sample.db"
# check before connecting, which implicitly create
db_exists = db_path.exists()
db_conn = sqlite3.connect(db_path)
if not db_exists:
    logger.info(f"{db_path} does not exist, initializing...")
    execute_template_sql(db_conn, env, "create.sql", params)

# UPSERT data
# restructure as list of tuples
flights = [(json.dumps(flight),) for flight in responses]
# db_conn.executemany(f"INSERT OR REPLACE INTO {TBL_NAME} ({JSON_COL}) VALUES( ? )", flights)
execute_template_sql(db_conn, env, "insert.sql", params, flights)

2023/10/24 10:51:32 [DEBUG] execute_template_sql: rendered SQL:
INSERT OR REPLACE INTO import_flight_records (flights_json) VALUES ( ? );


Testing if the transaction was successful

In [16]:
str_date = "2023-10-23"
with db_conn:
    res = db_conn.execute(
        f"""
        select count(*) num_flight, avg(arr_delay) avg_delay 
        from import_flight_records 
        where date(datetime(arr_time)) = '{str_date}'
        """
    )

# cannot subscript the returned cursor
print(next(res))

(1204, 21.625)


## Tweeting it out

In [17]:
def dict_factory(cursor, row):
    """
    cursor: sqlite3 cursor object
    row: tuple from query result
    returns the tuple row as dict
    """
    # .description attr returns a 7-tuple; only 1st is the col name
    fields = [descr[0] for descr in cursor.description]
    return {field: val for field, val in zip(fields, row)}


db_conn.row_factory = dict_factory

In [21]:
logger.debug("Querying database...")
params = dict(tbl_name=TBL_NAME, json_col=JSON_COL, str_date=str_date)
agg = execute_template_sql(db_conn, env, "agg.sql", params)
delays = execute_template_sql(db_conn, env, "delayed_json.sql", params)

2023/10/24 11:56:46 [DEBUG] <module>: Querying database...
2023/10/24 11:56:46 [DEBUG] execute_template_sql: rendered SQL:
WITH RECURSIVE 
    t(flight_date, total) AS (
        SELECT 
            DATE(DATETIME(arr_time)) flight_date,
            COUNT(*) total 
        FROM import_flight_records
        WHERE flight_date = '2023-10-23')
SELECT 
    t.total total,
    COUNT(arr_delay) num_delayed,
    AVG(CAST(arr_delay AS INTEGER)) avg_delay
FROM import_flight_records d LEFT JOIN t
WHERE DATE(d.arr_time) = t.flight_date
AND arr_delay > 0;
2023/10/24 11:56:46 [DEBUG] execute_template_sql: rendered SQL:
WITH RECURSIVE 
delays(flight_num, start, dest, delay) as (
    SELECT
        flight_iata_number as flight_num,
        json_extract(flights_json,'$.departure.airport') as start,
        json_extract(flights_json,'$.arrival.airport') as dest,
        CAST(arr_delay AS INTEGER) as delay
    FROM import_flight_records
    WHERE json_extract(flights_json,'$.flight_date') = '2023-10-23'
)


In [19]:
print(next(agg).values())

dict_values([1204, 504, 21.625])


In [20]:
for d in delays:
    print(d)

{'flight_num': 'MH6204', 'start': 'Sydney Kingsford Smith', 'dest': 'Kuala Lumpur (klia)', 'delay': 288}
{'flight_num': 'MH5656', 'start': 'Kuala Lumpur (klia)', 'dest': 'Schiphol', 'delay': 221}
{'flight_num': 'MH377', 'start': 'Guangzhou Baiyun', 'dest': 'Kuala Lumpur (klia)', 'delay': 186}


In [36]:
%load_ext sql

In [37]:
%sql sqlite:///../data/sample.db

In [39]:
%%sql
select
    count(*) num_flights,
    avg(cast(arr_delay as integer)) as avgdelay
from import_flight_records
where date(arr_time) = '2023-10-22'
and arr_delay is not null


 * sqlite:///../data/sample.db
Done.


num_flights,avgdelay
246,20.109756097560975


## 3. Tweet

In [22]:
total, num_delay, avg_delay = next(agg).values()
pt1 = f"{num_delay}/{total} MH flights were late on {str_date}"
pt2 = f"by an average of {avg_delay:.0f} min."
delays_in_sentences = "\n" + "\n".join(
    [
        f"{i+1} {d['flight_num']}: {d['start']} to {d['dest']}, {d['delay']} min"
        for i, d in enumerate(delays)
    ]
)
tweet = " ".join([pt1, pt2, delays_in_sentences])
if (tweet_chars := len(tweet)) > 280:
    logging.warning(f"Truncating tweet from {tweet_chars} to 280 chars")
    tweet = tweet[:280]
logger.debug(f"tweet length: {len(tweet)}")
# payload = write_flight_tweet(db_conn, str_date=str_date, template_dir=template_dir)

2023/10/24 11:57:09 [DEBUG] <module>: tweet length: 243


In [23]:
# tweet
oauth1_client = tweepy.Client(
    consumer_key=TWITTER_API_KEY,
    consumer_secret=TWITTER_API_SECRET,
    access_token=TWITTER_ACCESS_TOKEN,
    access_token_secret=TWITTER_ACCESS_SECRET,
)
local_tweet = True
if local_tweet:
    logger.info(f"offline tweet:\n{tweet}")
else:
    try:
        t_response = oauth1_client.create_tweet(text=tweet, user_auth=True)
        logger.info(f"link: https://twitter.com/user/status/{t_response.data['id']}")
        logger.info(f"text: {t_response.data['text']}")
    except Exception as e:
        logger.error(f"Tweet failed: {e}")

2023/10/24 11:57:49 [INFO] <module>: offline tweet:
504/1204 MH flights were late on 2023-10-23 by an average of 22 min. 
1 MH6204: Sydney Kingsford Smith to Kuala Lumpur (klia), 288 min
2 MH5656: Kuala Lumpur (klia) to Schiphol, 221 min
3 MH377: Guangzhou Baiyun to Kuala Lumpur (klia), 186 min


## Flight dates and Scheduling

1. Schedule the script to execute at midnight UTC
2. Request API
3. Look for entries from yesterday, i.e. `flight_date = date.today - 1`; this will include all flights that have left already, but will also include those that have not yet landed, or even some that have not yet departed if it was delayed. Assuming those are the minority, the script will only use results from this set

In [25]:
def sub(**kwargs):
    """
    sub will use all other args passed from main
    """
    b = kwargs["b"]
    c = kwargs["c"]
    logger.info(f"sub: {b} and {c}")


def main(a, *args, **kwargs):
    """
    main will only use a, pass on rest of inner funcs
    """
    logger.info(f"main: {a}")
    logger.info(f"{kwargs}")
    sub(**kwargs)


main(1, "foo", "bar", b=2, c=3)

2023/11/22 11:08:55 [INFO] main: main: 1
2023/11/22 11:08:55 [INFO] main: {'b': 2, 'c': 3}
2023/11/22 11:08:55 [INFO] sub: sub: 2 and 3


In [21]:
kw = {'b': 2, 'c': 3}
for k in kw: